In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./archive\img0.jpg
./archive\img0.txt
./archive\img1.jpg
./archive\img1.txt
./archive\img10.jpg
./archive\img10.txt
./archive\img100.jpg
./archive\img100.txt
./archive\img1000.txt
./archive\img1001.jpg
./archive\img1001.txt
./archive\img1002.jpg
./archive\img1002.txt
./archive\img1003.txt
./archive\img1004.jpg
./archive\img1004.txt
./archive\img1005.jpg
./archive\img1005.txt
./archive\img1006.txt
./archive\img1007.jpg
./archive\img1007.txt
./archive\img1008.txt
./archive\img1009.jpg
./archive\img1009.txt
./archive\img101.jpg
./archive\img101.txt
./archive\img1010.jpg
./archive\img1010.txt
./archive\img1011.jpg
./archive\img1011.txt
./archive\img1012.txt
./archive\img1013.jpg
./archive\img1013.txt
./archive\img1014.jpg
./archive\img1014.txt
./archive\img1015.txt
./archive\img1016.jpg
./archive\img1016.txt
./archive\img1017.txt
./archive\img1018.txt
./archive\img1019.jpg
./archive\img1019.txt
./archive\img102.jpg
./archive\img102.txt
./archive\img1020.txt
./archive\img1021.jpg
./archive\

In [2]:
#!pip install roboflow

In [3]:
data_dir = './archive'
image_label_list = []
valid_exts = ['.png', '.jpg', '.jpeg']

for fname in os.listdir(data_dir):
    if fname.endswith('.txt'):
        base_name = fname.replace('.txt', '')

        # Find the actual existing image extension
        matched_img = None
        for ext in valid_exts:
            candidate = os.path.join(data_dir, base_name + ext)
            if os.path.exists(candidate):
                matched_img = candidate
                break
        
        # Skip if no image is found
        if matched_img is None:
            continue

        # Read the label and extract only the most severe class
        txt_path = os.path.join(data_dir, fname)
        with open(txt_path, 'r') as f:
            class_ids = [int(line.split()[0]) for line in f.readlines()]
        
        if class_ids:
            max_class = max(class_ids)
            image_label_list.append((matched_img, max_class))

print("Total number of images:", len(image_label_list))
print("Example:", image_label_list[:5])

Total number of images: 1223
Example: [('./archive\\img0.jpg', 0), ('./archive\\img1.jpg', 1), ('./archive\\img10.jpg', 0), ('./archive\\img100.jpg', 0), ('./archive\\img1001.jpg', 0)]


In [4]:
import os
import pandas as pd

rf_base_dir = "./Skin-Burns--2"
splits = ["train", "valid", "test"]

# Integrate Roboflow data
for split in splits:
    split_dir = os.path.join(rf_base_dir, split)
    csv_path = os.path.join(split_dir, "_classes.csv")

    if not os.path.exists(csv_path):
        continue

    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()  # Strip whitespace from column names

    for _, row in df.iterrows():
        fname = row['filename'].strip()
        img_path = os.path.join(split_dir, fname)

        if not fname.lower().endswith('.jpg'):
            continue

        if not os.path.exists(img_path):
            continue

        # Extract the class label (the index with the highest value among '0', '1', '2')
        label = int(row[['0', '1', '2']].astype(int).idxmax())
        image_label_list.append((img_path, label))

print("Total number of images:", len(image_label_list))
print("Example:", image_label_list[:5])

Total number of images: 3848
Example: [('./archive\\img0.jpg', 0), ('./archive\\img1.jpg', 1), ('./archive\\img10.jpg', 0), ('./archive\\img100.jpg', 0), ('./archive\\img1001.jpg', 0)]


In [5]:
from sklearn.model_selection import train_test_split

# 8:2 split
train_list, val_list = train_test_split(
    image_label_list,
    test_size=0.2,
    stratify=[label for _, label in image_label_list],  # 클래스 비율 유지
    random_state=42
)

In [6]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class SkinBurnDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data = data_list
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label
    
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [7]:
from torch.utils.data import DataLoader

train_dataset = SkinBurnDataset(train_list, transform=transform_train)
val_dataset = SkinBurnDataset(val_list, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

print(f"Train loader: {len(train_loader)} batches")
print(f"Val loader: {len(val_loader)} batches")

Train loader: 97 batches
Val loader: 25 batches


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm # 👈 1. 匯入 tqdm
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn
import torch
# ... (model definition part)
print("準備建立模型...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("準備下載模型權重...")
weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights)
print("模型權重下載完成！")

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 3)

model = model.to(device)
print("模型已載入到 device。準備開始訓練...")

# ... (training loop part)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # 👇 2. 將 train_loader 用 tqdm 包起來
    train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

    for images, labels in train_progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # (選用) 在進度條後面顯示即時 loss
        train_progress_bar.set_postfix(loss=loss.item())

    train_acc = 100 * correct / total
    avg_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    
    # 👇 3. 將 val_loader 用 tqdm 包起來
    val_progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
    
    with torch.no_grad():
        for images, labels in val_progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}]  "
          f"Loss: {avg_loss:.4f}  "
          f"Train Acc: {train_acc:.2f}%  "
          f"Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")
        print(f"✨ New best model saved with Val Acc: {best_val_acc:.2f}%")

準備建立模型...
準備下載模型權重...
模型權重下載完成！
模型已載入到 device。準備開始訓練...


Epoch 1/10 [Train]:   0%|          | 0/97 [00:00<?, ?it/s]

Epoch 1/10 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/10]  Loss: 0.7809  Train Acc: 67.19%  Val Acc: 84.81%
✨ New best model saved with Val Acc: 84.81%


Epoch 2/10 [Train]:   0%|          | 0/97 [00:00<?, ?it/s]

...

Epoch 9/10 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [9/10]  Loss: 0.1138  Train Acc: 96.07%  Val Acc: 95.06%


Epoch 10/10 [Train]:   0%|          | 0/97 [00:00<?, ?it/s]

Epoch 10/10 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [10/10]  Loss: 0.0964  Train Acc: 96.69%  Val Acc: 95.32%
✨ New best model saved with Val Acc: 95.32%
